In [1]:
%cd Raindrop/code/baselines

/../ML4ED/Raindrop/code/baselines


In [2]:
import numpy as np

wandb = False

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, average_precision_score, precision_score, recall_score, f1_score
from models import TransformerModel, TransformerModel2, SEFT
from utils_phy12 import *

from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
import pandas as pd

/opt/conda/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Transformer

In [3]:
MOOCs_list = [
'villesafricaines_002.csv',
 'villesafricaines_003.csv',
 'microcontroleurs_004.csv',
 'dsp_004.csv',
 'hwts_001.csv',
 'dsp_001.csv',
 'progfun_002.csv',
 'microcontroleurs_003.csv',
 'geomatique_003.csv',
 'villesafricaines_001.csv',
 'progfun_003.csv',
 'dsp_002.csv',
 'structures_002.csv',
 'initprogcpp_001.csv',
 'analysenumerique_003.csv',
 'microcontroleurs_006.csv',
 'dsp_005.csv',
 'hwts_002.csv',
 'dsp_006.csv',
 'analysenumerique_002.csv',
 'structures_003.csv',
 'microcontroleurs_005.csv',
 'venture_001.csv',
 'analysenumerique_001.csv',
 'cpp_fr_001.csv',
 'structures_001.csv'
]
MOOCs_list = [i.replace("_", "-").split('.')[0] for i in MOOCs_list]

dims4 = [12,
 12,
 13,
 12,
 12,
 6,
 12,
 13,
 12,
 12,
 12,
 12,
 13,
 13,
 6,
 13,
 12,
 12,
 12,
 12,
 13,
 13,
 12,
 6,
 13,
 12]

dims6 = [12,
 12,
 13,
 12,
 12,
 6,
 12,
 13,
 12,
 12,
 12,
 12,
 13,
 13,
 6,
 13,
 12,
 12,
 13,
 12,
 13,
 13,
 12,
 6,
 13,
 12]
dims = {40: dims4, 60: dims6}
data_path = '/../data'
percentile = 40

In [4]:

torch.manual_seed(0)



imputation = 'no_imputation'

arch = 'standard'

model_path = '../../models/transformer/'

dataset = 'P12'


base_path = '/../data/prep_data'
data_path = '/../data'



def one_hot(y_):
    y_ = y_.reshape(len(y_))
    y_ = [int(x) for x in y_]
    n_values = np.max(y_) + 1
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]

feature_removal_level = 'no_removal'


for MOOC_idx, MOOC in enumerate(MOOCs_list):
    
    for percentile in [40, 60]:
        
        
        d_inp = dims[percentile][MOOC_idx]




        missing_ratio = 0


        num_epochs = 20
        learning_rate = 0.001


        d_static = 9

        static_info = 0


        d_model = d_inp
        nhid = 2 * d_model
        nlayers = 1
        nhead = 1

        dropout = 0.3


        max_len = 1000
        n_classes = 2


        aggreg = 'mean'

        MAX = 100

        n_runs = 1
        n_splits = 1
        subset = False

        split = 'random'
        reverse = False
        baseline = True

        acc_arr = np.zeros((n_splits, n_runs))
        auprc_arr = np.zeros((n_splits, n_runs))
        auroc_arr = np.zeros((n_splits, n_runs))
        precision_arr = np.zeros((n_splits, n_runs))
        recall_arr = np.zeros((n_splits, n_runs))
        F1_arr = np.zeros((n_splits, n_runs))

        for k in range(n_splits):
            split_idx = k + 1
            print('Split id: %d' % split_idx)

            #if dataset == 'P12':
            #    if subset == True:
            #        split_path = '/splits/phy12_split_subset' + str(split_idx) + '.npy'
            #    else:
            #        split_path = '/splits/phy12_split' + str(split_idx) + '.npy'
            #elif dataset == 'P19':
            #    split_path = '/splits/phy19_split' + str(split_idx) + '_new.npy'
            #elif dataset == 'eICU':
            #    split_path = '/splits/eICU_split' + str(split_idx) + '.npy'
            #elif dataset == 'PAM':
            #    split_path = '/splits/PAM_split_' + str(split_idx) + '.npy'


            # prepare the data:
            #Ptrain, Pval, Ptest, ytrain, yval, ytest = get_data_split(base_path, split_path, split_type=split,
            #                                                          reverse=reverse, baseline=baseline, dataset=dataset,
            #                                                          predictive_label=args.predictive_label)
            Pdict_list = np.load(os.path.join(base_path, f"{MOOC}_{percentile}_data_hard_fail.npy"), allow_pickle=True)
            arr_outcomes = np.load(os.path.join(base_path, f"{MOOC}_{percentile}_y_hard_fail.npy"), allow_pickle=True)

            #Ptrain, Ptest, ytrain, ytest = train_test_split(Pdict_list, arr_outcomes, test_size=0.1, random_state=1)
            #Ptrain, Pval, ytrain, yval = train_test_split(Ptrain, ytrain, test_size=1/9, random_state=1)
            args_train, args_val, args_test = np.load(os.path.join(data_path, 
                                                                           'split_args', f"split_{MOOC.replace('-', '_')}.npy"),
                                                             allow_pickle=True)
            Ptrain = Pdict_list[args_train]
            Pval = Pdict_list[args_val]
            Ptest = Pdict_list[args_test]
            ytrain = arr_outcomes[args_train, :]
            yval = arr_outcomes[args_val, :]
            ytest = arr_outcomes[args_test, :]


            zero_indices = [i for i, item in enumerate(Ptrain) if item['length'] == 0]
            #zero_Ptrain = Ptrain[zero_indices]
            Ptrain = np.delete(Ptrain, zero_indices, axis=0)
            ytrain = np.delete(ytrain, zero_indices, axis=0)

            zero_indices = [i for i, item in enumerate(Pval) if item['length'] == 0]
            zero_yval = yval[zero_indices]
            Pval = np.delete(Pval, zero_indices, axis=0)
            yval = np.delete(yval, zero_indices, axis=0)

            zero_indices = [i for i, item in enumerate(Ptest) if item['length'] == 0]
            zero_ytest = ytest[zero_indices]
            Ptest = np.delete(Ptest, zero_indices, axis=0)  
            ytest = np.delete(ytest, zero_indices, axis=0)

            print(len(Ptrain), len(Pval), len(Ptest), len(ytrain), len(yval), len(ytest))
            print(sum(ytrain), sum(yval), sum(ytest))

            # impute missing values
            if imputation != 'no_imputation':
                if dataset == 'P12' or dataset == 'P19' or dataset == 'eICU':
                    X_features_train = np.array([d['arr'] for d in Ptrain])
                    X_time_train = np.array([d['time'] for d in Ptrain])
                    X_features_val = np.array([d['arr'] for d in Pval])
                    X_time_val = np.array([d['time'] for d in Pval])
                    X_features_test = np.array([d['arr'] for d in Ptest])
                    X_time_test = np.array([d['time'] for d in Ptest])
                elif dataset == 'PAM':
                    X_features_train = Ptrain
                    X_time_train = np.array([np.arange(1, Ptrain.shape[1] + 1)[..., np.newaxis] for d in Ptrain])
                    X_features_val = Pval
                    X_time_val = np.array([np.arange(1, Pval.shape[1] + 1)[..., np.newaxis] for d in Pval])
                    X_features_test = Ptest
                    X_time_test = np.array([np.arange(1, Ptest.shape[1] + 1)[..., np.newaxis] for d in Ptest])

                if dataset == 'P12' or dataset == 'P19' or dataset == 'PAM':
                    missing_value_num = 0
                elif dataset == 'eICU':
                    missing_value_num = -1

                if imputation == 'mean':
                    features_means = get_features_mean(X_features_train)
                    X_features_train = mean_imputation(X_features_train, X_time_train, features_means, missing_value_num)
                    X_features_val = mean_imputation(X_features_val, X_time_val, features_means, missing_value_num)
                    X_features_test = mean_imputation(X_features_test, X_time_test, features_means, missing_value_num)
                elif imputation == 'forward':
                    X_features_train = forward_imputation(X_features_train, X_time_train, missing_value_num)
                    X_features_val = forward_imputation(X_features_val, X_time_val, missing_value_num)
                    X_features_test = forward_imputation(X_features_test, X_time_test, missing_value_num)
                elif imputation == 'cubic_spline':
                    X_features_train = cubic_spline_imputation(X_features_train, X_time_train, missing_value_num)
                    X_features_val = cubic_spline_imputation(X_features_val, X_time_val, missing_value_num)
                    X_features_test = cubic_spline_imputation(X_features_test, X_time_test, missing_value_num)

                if dataset == 'P12' or dataset == 'P19' or dataset == 'eICU':
                    for i, pat in enumerate(X_features_train):
                        Ptrain[i]['arr'] = pat
                    for i, pat in enumerate(X_features_val):
                        Pval[i]['arr'] = pat
                    for i, pat in enumerate(X_features_test):
                        Ptest[i]['arr'] = pat
                elif dataset == 'PAM':
                    for i, pat in enumerate(X_features_train):
                        Ptrain[i] = pat
                    for i, pat in enumerate(X_features_val):
                        Pval[i] = pat
                    for i, pat in enumerate(X_features_test):
                        Ptest[i] = pat

            if dataset == 'P12' or dataset == 'P19' or dataset == 'eICU':
                T, F = Ptrain[0]['arr'].shape
                D = len(Ptrain[0]['extended_static'])

                Ptrain_tensor = np.zeros((len(Ptrain), T, F))
                Ptrain_static_tensor = np.zeros((len(Ptrain), D))

                for i in range(len(Ptrain)):
                    Ptrain_tensor[i] = Ptrain[i]['arr']
                    Ptrain_static_tensor[i] = Ptrain[i]['extended_static']

                mf, stdf = getStats(Ptrain_tensor)
                ms, ss = getStats_static(Ptrain_static_tensor, dataset=dataset)

                Ptrain_tensor, Ptrain_static_tensor, Ptrain_time_tensor, ytrain_tensor = tensorize_normalize(Ptrain, ytrain,
                                                                                                             mf,
                                                                                                             stdf, ms, ss)
                Pval_tensor, Pval_static_tensor, Pval_time_tensor, yval_tensor = tensorize_normalize(Pval, yval, mf, stdf,
                                                                                                     ms, ss)
                Ptest_tensor, Ptest_static_tensor, Ptest_time_tensor, ytest_tensor = tensorize_normalize(Ptest, ytest, mf,
                                                                                                         stdf, ms,
                                                                                                         ss)

                print(Ptrain_tensor.shape, Ptrain_static_tensor.shape, Ptrain_time_tensor.shape, ytrain_tensor.shape)
            elif dataset == 'PAM':
                T, F = Ptrain[0].shape
                D = 1

                Ptrain_tensor = Ptrain
                Ptrain_static_tensor = np.zeros((len(Ptrain), D))

                mf, stdf = getStats(Ptrain)
                Ptrain_tensor, Ptrain_static_tensor, Ptrain_time_tensor, ytrain_tensor = tensorize_normalize_other(Ptrain, ytrain, mf, stdf)
                Pval_tensor, Pval_static_tensor, Pval_time_tensor, yval_tensor = tensorize_normalize_other(Pval, yval, mf, stdf)
                Ptest_tensor, Ptest_static_tensor, Ptest_time_tensor, ytest_tensor = tensorize_normalize_other(Ptest, ytest, mf, stdf)

            # remove part of variables in validation and test set
            if missing_ratio > 0:
                num_all_features = Pval_tensor.shape[2]
                num_missing_features = round(missing_ratio * num_all_features)
                if feature_removal_level == 'sample':
                    for i, patient in enumerate(Pval_tensor):
                        idx = np.random.choice(num_all_features, num_missing_features, replace=False)
                        patient[:, idx] = torch.zeros(Pval_tensor.shape[1], num_missing_features)  # values
                        Pval_tensor[i] = patient
                    for i, patient in enumerate(Ptest_tensor):
                        idx = np.random.choice(num_all_features, num_missing_features, replace=False)
                        patient[:, idx] = torch.zeros(Ptest_tensor.shape[1], num_missing_features)   # values
                        Ptest_tensor[i] = patient
                elif feature_removal_level == 'set':
                    density_score_indices = np.load('saved/IG_density_scores_' + dataset + '.npy', allow_pickle=True)[:, 0]
                    idx = density_score_indices[:num_missing_features].astype(int)
                    Pval_tensor[:, :, idx] = torch.zeros(Pval_tensor.shape[0], Pval_tensor.shape[1], num_missing_features)
                    Ptest_tensor[:, :, idx] = torch.zeros(Ptest_tensor.shape[0], Ptest_tensor.shape[1], num_missing_features)

            Ptrain_tensor = Ptrain_tensor.permute(1, 0, 2)
            Pval_tensor = Pval_tensor.permute(1, 0, 2)
            Ptest_tensor = Ptest_tensor.permute(1, 0, 2)

            Ptrain_time_tensor = Ptrain_time_tensor.squeeze(2).permute(1, 0)
            Pval_time_tensor = Pval_time_tensor.squeeze(2).permute(1, 0)
            Ptest_time_tensor = Ptest_time_tensor.squeeze(2).permute(1, 0)

            for m in range(n_runs):
                print('- - Run %d - -' % (m + 1))

                if dataset == 'P12' or dataset == 'P19' or dataset == 'eICU':
                    model = TransformerModel2(d_inp, d_model, nhead, nhid, nlayers, dropout, max_len,
                                              d_static, MAX, 0.5, aggreg, n_classes, static=False)
                elif dataset == 'PAM':
                    model = TransformerModel2(d_inp, d_model, nhead, nhid, nlayers, dropout, max_len,
                                              d_static, MAX, 0.5, aggreg, n_classes, static=False)

                model = model.cuda()

                criterion = torch.nn.CrossEntropyLoss().cuda()

                optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
                scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1,
                                                                       patience=1, threshold=0.0001, threshold_mode='rel',
                                                                       cooldown=0, min_lr=1e-8, eps=1e-08, verbose=True)

                idx_0 = np.where(ytrain == 0)[0]
                idx_1 = np.where(ytrain == 1)[0]

                if dataset == 'P12' or dataset == 'P19' or dataset == 'eICU':
                    strategy = 2
                elif dataset == 'PAM':
                    strategy = 3

                n0, n1 = len(idx_0), len(idx_1)
                expanded_idx_1 = np.concatenate([idx_1, idx_1, idx_1], axis=0)
                expanded_n1 = len(expanded_idx_1)

                batch_size = 128
                if strategy == 1:
                    n_batches = 10
                elif strategy == 2:
                    K0 = n0 // int(batch_size / 2)
                    K1 = expanded_n1 // int(batch_size / 2)
                    n_batches = np.min([K0, K1])
                elif strategy == 3:
                    n_batches = 30

                best_aupr_val = best_auc_val = 0.0

                start = time.time()
                if wandb:
                    wandb.watch(model)
                for epoch in range(num_epochs):
                    model.train()

                    if strategy == 2:
                        np.random.shuffle(expanded_idx_1)
                        I1 = expanded_idx_1
                        np.random.shuffle(idx_0)
                        I0 = idx_0

                    for n in range(n_batches):
                        if strategy == 1:
                            idx = random_sample(idx_0, idx_1, batch_size)
                        elif strategy == 2:
                            """In each batch=128, 64 positive samples, 64 negative samples"""
                            idx0_batch = I0[n * int(batch_size / 2):(n + 1) * int(batch_size / 2)]
                            idx1_batch = I1[n * int(batch_size / 2):(n + 1) * int(batch_size / 2)]
                            idx = np.concatenate([idx0_batch, idx1_batch], axis=0)
                        elif strategy == 3:
                            idx = np.random.choice(list(range(Ptrain_tensor.shape[1])), size=int(batch_size), replace=False)
                            # idx = random_sample_8(ytrain, batch_size)   # to balance dataset

                        if dataset == 'P12' or dataset == 'P19' or dataset == 'eICU':
                            P, Ptime, Pstatic, y = Ptrain_tensor[:, idx, :].cuda(), Ptrain_time_tensor[:, idx].cuda(), \
                                                   Ptrain_static_tensor[idx].cuda(), ytrain_tensor[idx].cuda()
                        elif dataset == 'PAM':
                            P, Ptime, Pstatic, y = Ptrain_tensor[:, idx, :].cuda(), Ptrain_time_tensor[:, idx].cuda(), \
                                                   None, ytrain_tensor[idx].cuda()

                        lengths = torch.sum(Ptime > 0, dim=0)

                        #print(P)
                        #print(Ptime)
                        #print(static_info)
                        #print(Pstatic)
                        #print(Pstatic.shape)

                        outputs = evaluate_standard(model, P, Ptime, Pstatic, static=None)

                        #print(outputs)

                        optimizer.zero_grad()
                        loss = criterion(outputs, y)
                        loss.backward()
                        optimizer.step()

                    if dataset == 'P12' or dataset == 'P19' or dataset == 'eICU':
                        train_probs = torch.squeeze(torch.sigmoid(outputs))
                        train_probs = train_probs.cpu().detach().numpy()
                        train_y = y.cpu().detach().numpy()
                        train_auroc = roc_auc_score(train_y, train_probs[:, 1])
                        train_auprc = average_precision_score(train_y, train_probs[:, 1])
                    elif dataset == 'PAM':
                        train_probs = torch.squeeze(nn.functional.softmax(outputs, dim=1))
                        train_probs = train_probs.cpu().detach().numpy()
                        train_y = y.cpu().detach().numpy()
                        train_auroc = roc_auc_score(one_hot(train_y), train_probs)
                        train_auprc = average_precision_score(one_hot(train_y), train_probs)

                    if wandb:
                        wandb.log({ "train_loss": loss.item(), "train_auprc": train_auprc, "train_auroc": train_auroc})

                    """Validation"""
                    model.eval()
                    if epoch ==0 or epoch % 1 == 0:
                        with torch.no_grad():
                            out_val = evaluate_standard(model, Pval_tensor, Pval_time_tensor, Pval_static_tensor, static=static_info)
                            out_val = torch.squeeze(torch.sigmoid(out_val))
                            out_val = out_val.detach().cpu().numpy()

                            val_loss = criterion(torch.from_numpy(out_val), torch.from_numpy(yval.squeeze(1)).long())

                            if dataset == 'P12' or dataset == 'P19' or dataset == 'eICU':
                                auc_val = roc_auc_score(yval, out_val[:, 1])
                                aupr_val = average_precision_score(yval, out_val[:, 1])
                            elif dataset == 'PAM':
                                auc_val = roc_auc_score(one_hot(yval), out_val)
                                aupr_val = average_precision_score(one_hot(yval), out_val)

                            print("Validation: Epoch %d,  val_loss:%.4f, aupr_val: %.2f, auc_val: %.2f" % (epoch,
                              val_loss.item(), aupr_val * 100, auc_val * 100))

                            if wandb:
                                wandb.log({ "val_loss": val_loss.item(), "val_auprc": aupr_val, "val_auroc": auc_val})

                            scheduler.step(aupr_val)
                            if auc_val > best_auc_val:
                                best_auc_val = auc_val
                                print(
                                    "**[S] Epoch %d, aupr_val: %.4f, auc_val: %.4f **" % (epoch, aupr_val * 100, auc_val * 100))
                                torch.save(model.state_dict(), model_path + arch + '_' + str(split_idx) + '.pt')

                end = time.time()
                time_elapsed = end - start
                print('Total Time elapsed: %.3f mins' % (time_elapsed / 60.0))

                """Testing"""
                model.load_state_dict(torch.load(model_path + arch + '_' + str(split_idx) + '.pt'))
                model.eval()

                with torch.no_grad():
                    out_test = evaluate(model, Ptest_tensor, Ptest_time_tensor, Ptest_static_tensor, n_classes=n_classes, static=None).numpy()
                    ypred = np.argmax(out_test, axis=1)

                    # Adding zero interaction students
                    ytest = np.append(ytest, zero_ytest, axis=0)
                    ypred = np.append(ypred, np.zeros([1, len(zero_ytest)]))

                    denoms = np.sum(np.exp(out_test), axis=1).reshape((-1, 1))
                    probs = np.exp(out_test) / denoms
                    
                    # Adding zero interaction students
                    probs = np.append(probs, np.zeros([len(zero_ytest), 2]), axis=0)

                    acc = np.sum(ytest.ravel() == ypred.ravel()) / ytest.shape[0]
                    bac = balanced_accuracy_score(ytest.ravel(), ypred.ravel())


                    auc = roc_auc_score(ytest, probs[:, 1])
                    aupr = average_precision_score(ytest, probs[:, 1])


                # store
                    results = pd.DataFrame(columns=['course', 'percentile', 'acc', 'bac', 'auc', 'auprc'])
                    results.loc[0] = [MOOC, percentile, acc, bac, auc, aupr]
                    results.to_csv(f"../../transformer_results/test_{MOOC}_{percentile}.csv")
                    print(results)

                    
                    
                    #VALIDATION
                    out_val = evaluate(model, Pval_tensor, Pval_time_tensor, Pval_static_tensor, n_classes=n_classes, static=None).numpy()
                    ypred = np.argmax(out_val, axis=1)

                    # Adding zero interaction students
                    yval = np.append(yval, zero_yval, axis=0)
                    ypred = np.append(ypred, np.zeros([1, len(zero_yval)]))

                    denoms = np.sum(np.exp(out_val), axis=1).reshape((-1, 1))
                    probs = np.exp(out_val) / denoms
                    
                    # Adding zero interaction students
                    probs = np.append(probs, np.zeros([len(zero_yval), 2]), axis=0)

                    acc = np.sum(yval.ravel() == ypred.ravel()) / yval.shape[0]
                    bac = balanced_accuracy_score(yval.ravel(), ypred.ravel())


                    auc = roc_auc_score(yval, probs[:, 1])
                    aupr = average_precision_score(yval, probs[:, 1])


                # store
                    results_val = pd.DataFrame(columns=['course', 'percentile', 'acc', 'bac', 'auc', 'auprc'])
                    results_val.loc[0] = [MOOC, percentile, acc, bac, auc, aupr]
                    results_val.to_csv(f"../../transformer_results/val_{MOOC}_{percentile}.csv")
                    print(results_val)


Split id: 1
1966 234 247 1966 234 247
[176] [21] [24]


/opt/conda/envs/py39/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/opt/conda/envs/py39/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/envs/py39/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/conda/envs/py39/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/conda/envs/py39/lib/python3.9/site-packages/numpy/core/_metho

torch.Size([1966, 1000, 24]) torch.Size([1966, 9]) torch.Size([1966, 1000, 1]) torch.Size([1966])
- - Run 1 - -
Validation: Epoch 0,  val_loss:0.6899, aupr_val: 23.17, auc_val: 66.35
**[S] Epoch 0, aupr_val: 23.1653, auc_val: 66.3537 **
Validation: Epoch 1,  val_loss:0.6917, aupr_val: 20.19, auc_val: 66.91
**[S] Epoch 1, aupr_val: 20.1857, auc_val: 66.9126 **
Validation: Epoch 2,  val_loss:0.6963, aupr_val: 25.06, auc_val: 72.70
**[S] Epoch 2, aupr_val: 25.0593, auc_val: 72.7029 **
Validation: Epoch 3,  val_loss:0.6878, aupr_val: 33.20, auc_val: 79.25
**[S] Epoch 3, aupr_val: 33.2033, auc_val: 79.2533 **
Validation: Epoch 4,  val_loss:0.6899, aupr_val: 33.29, auc_val: 79.83
**[S] Epoch 4, aupr_val: 33.2917, auc_val: 79.8346 **
Validation: Epoch 5,  val_loss:0.6781, aupr_val: 37.46, auc_val: 82.88
**[S] Epoch 5, aupr_val: 37.4617, auc_val: 82.8750 **
Validation: Epoch 6,  val_loss:0.6625, aupr_val: 37.42, auc_val: 82.86
Validation: Epoch 7,  val_loss:0.6665, aupr_val: 37.46, auc_val: 82

/opt/conda/envs/py39/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/opt/conda/envs/py39/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/envs/py39/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/conda/envs/py39/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/conda/envs/py39/lib/python3.9/site-packages/numpy/core/_metho

torch.Size([2125, 1000, 24]) torch.Size([2125, 9]) torch.Size([2125, 1000, 1]) torch.Size([2125])
- - Run 1 - -
Validation: Epoch 0,  val_loss:0.6798, aupr_val: 43.51, auc_val: 79.00
**[S] Epoch 0, aupr_val: 43.5120, auc_val: 79.0036 **
Validation: Epoch 1,  val_loss:0.6839, aupr_val: 47.85, auc_val: 82.48
**[S] Epoch 1, aupr_val: 47.8467, auc_val: 82.4819 **
Validation: Epoch 2,  val_loss:0.6783, aupr_val: 48.21, auc_val: 82.41
Validation: Epoch 3,  val_loss:0.6688, aupr_val: 46.52, auc_val: 82.19
Validation: Epoch 4,  val_loss:0.6685, aupr_val: 46.52, auc_val: 82.19
Epoch 00005: reducing learning rate of group 0 to 1.0000e-04.
Validation: Epoch 5,  val_loss:0.6685, aupr_val: 46.52, auc_val: 82.19
Validation: Epoch 6,  val_loss:0.6670, aupr_val: 46.52, auc_val: 82.19
Epoch 00007: reducing learning rate of group 0 to 1.0000e-05.
Validation: Epoch 7,  val_loss:0.6669, aupr_val: 46.52, auc_val: 82.19
Validation: Epoch 8,  val_loss:0.6669, aupr_val: 46.52, auc_val: 82.19
Epoch 00009: redu

...

torch.Size([595, 1000, 26]) torch.Size([595, 9]) torch.Size([595, 1000, 1]) torch.Size([595])
- - Run 1 - -
Validation: Epoch 0,  val_loss:0.6899, aupr_val: 37.12, auc_val: 47.45
**[S] Epoch 0, aupr_val: 37.1197, auc_val: 47.4497 **
Validation: Epoch 1,  val_loss:0.6892, aupr_val: 37.10, auc_val: 47.38
Validation: Epoch 2,  val_loss:0.6917, aupr_val: 37.10, auc_val: 47.38
Epoch 00003: reducing learning rate of group 0 to 1.0000e-04.
Validation: Epoch 3,  val_loss:0.6919, aupr_val: 37.10, auc_val: 47.38
Validation: Epoch 4,  val_loss:0.6922, aupr_val: 37.10, auc_val: 47.38
Epoch 00005: reducing learning rate of group 0 to 1.0000e-05.
Validation: Epoch 5,  val_loss:0.6922, aupr_val: 37.10, auc_val: 47.38
Validation: Epoch 6,  val_loss:0.6922, aupr_val: 37.10, auc_val: 47.38
Epoch 00007: reducing learning rate of group 0 to 1.0000e-06.
Validation: Epoch 7,  val_loss:0.6922, aupr_val: 37.10, auc_val: 47.38
Validation: Epoch 8,  val_loss:0.6922, aupr_val: 37.10, auc_val: 47.38
Epoch 00009: 

/opt/conda/envs/py39/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/opt/conda/envs/py39/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/envs/py39/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/conda/envs/py39/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/conda/envs/py39/lib/python3.9/site-packages/numpy/core/_metho

torch.Size([626, 1000, 26]) torch.Size([626, 9]) torch.Size([626, 1000, 1]) torch.Size([626])
- - Run 1 - -
Validation: Epoch 0,  val_loss:0.6870, aupr_val: 44.78, auc_val: 48.94
**[S] Epoch 0, aupr_val: 44.7788, auc_val: 48.9444 **
Validation: Epoch 1,  val_loss:0.6839, aupr_val: 44.49, auc_val: 48.66
Validation: Epoch 2,  val_loss:0.6879, aupr_val: 44.43, auc_val: 48.52
Epoch 00003: reducing learning rate of group 0 to 1.0000e-04.
Validation: Epoch 3,  val_loss:0.6883, aupr_val: 44.35, auc_val: 48.31
Validation: Epoch 4,  val_loss:0.6883, aupr_val: 44.53, auc_val: 48.73
Epoch 00005: reducing learning rate of group 0 to 1.0000e-05.
Validation: Epoch 5,  val_loss:0.6883, aupr_val: 44.53, auc_val: 48.73
Validation: Epoch 6,  val_loss:0.6883, aupr_val: 44.53, auc_val: 48.73
Epoch 00007: reducing learning rate of group 0 to 1.0000e-06.
Validation: Epoch 7,  val_loss:0.6883, aupr_val: 44.53, auc_val: 48.73
Validation: Epoch 8,  val_loss:0.6883, aupr_val: 44.53, auc_val: 48.73
Epoch 00009: 

/opt/conda/envs/py39/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/opt/conda/envs/py39/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/envs/py39/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/conda/envs/py39/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/conda/envs/py39/lib/python3.9/site-packages/numpy/core/_metho

torch.Size([75, 1000, 24]) torch.Size([75, 9]) torch.Size([75, 1000, 1]) torch.Size([75])
- - Run 1 - -
Validation: Epoch 0,  val_loss:0.7015, aupr_val: 59.31, auc_val: 27.78
**[S] Epoch 0, aupr_val: 59.3056, auc_val: 27.7778 **
Validation: Epoch 1,  val_loss:0.7015, aupr_val: 59.31, auc_val: 27.78
Validation: Epoch 2,  val_loss:0.7015, aupr_val: 59.31, auc_val: 27.78
Epoch 00003: reducing learning rate of group 0 to 1.0000e-04.
Validation: Epoch 3,  val_loss:0.7015, aupr_val: 59.31, auc_val: 27.78
Validation: Epoch 4,  val_loss:0.7015, aupr_val: 59.31, auc_val: 27.78
Epoch 00005: reducing learning rate of group 0 to 1.0000e-05.
Validation: Epoch 5,  val_loss:0.7015, aupr_val: 59.31, auc_val: 27.78
Validation: Epoch 6,  val_loss:0.7015, aupr_val: 59.31, auc_val: 27.78
Epoch 00007: reducing learning rate of group 0 to 1.0000e-06.
Validation: Epoch 7,  val_loss:0.7015, aupr_val: 59.31, auc_val: 27.78
Validation: Epoch 8,  val_loss:0.7015, aupr_val: 59.31, auc_val: 27.78
Epoch 00009: redu